<a href="https://colab.research.google.com/github/himanshu11081999/Antimicrobial_PPS/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================== #
# Prescription Extraction Script (Google Colab)
# Customized for: /content/images/archive
# ========================== #

# 1. Install additional libraries if needed
!pip install kagglehub -q
!pip install sentencepiece -q

# 2. Import libraries
import os
import json
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch
import zipfile

# 3. Upload your archive.zip manually
# In Colab: Left side > Files > Upload archive.zip manually.

# 4. Unzip the archive
with zipfile.ZipFile('/content/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/images')  # Extract into 'images' folder

# 5. Update the correct image folder path (adjusted here!)
image_folder = '/content/images'  # Corrected path - assuming images were extracted directly to 'images'

print("✅ Image folder set to:", image_folder)

# 6. Load Donut processor and model
print("Loading Donut model...")
model_name = "naver-clova-ix/donut-base-finetuned-docvqa"
processor = DonutProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# 7. Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()
torch.set_grad_enabled(False)

# 8. Extraction function
def extract_from_prescription(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    generated_ids = model.generate(pixel_values, max_length=512, num_beams=3)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    try:
        structured_output = json.loads(generated_text)
    except json.JSONDecodeError:
        structured_output = {"raw_text": generated_text}

    return structured_output

# 9. Process all images
output_data = {}

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

print(f"🔍 Found {len(image_files)} image(s) to process.")

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    print(f"Processing: {image_file}")
    structured_output = extract_from_prescription(image_path)
    output_data[image_file] = structured_output

# 10. Save extracted outputs
output_filename = "extracted_prescriptions_colab.json"
with open(output_filename, "w") as f:
    json.dump(output_data, f, indent=4)

print(f"\n✅ Extraction completed! Results saved to: {output_filename}")


✅ Image folder set to: /content/images
Loading Donut model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/803M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cr

🔍 Found 129 image(s) to process.
Processing: 119.jpg
